# Python 地理信息数据可视化

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

import matplotlib.pyplot as plt

pd.options.display.max_rows = 10

In [ ]:
countries = geopandas.read_file("zip://./data/ne_110m_admin_0_countries.zip")
cities = geopandas.read_file("zip://./data/ne_110m_populated_places.zip")
rivers = geopandas.read_file("zip://./data/ne_50m_rivers_lake_centerlines.zip")

## GeoPandas 可视化功能

#### 基本图表

In [ ]:
countries.plot()

#### 调整尺寸

In [ ]:
countries.plot(figsize=(15, 15))

#### 移除盒子 / x 和 y 坐标标签

In [ ]:
ax = countries.plot(figsize=(15, 15))
ax.set_axis_off()

#### 基于列的数值填色

我们先用人均 GDP 创建一列:

In [ ]:
countries = countries[(countries['pop_est'] >0 ) & (countries['name'] != "Antarctica")]

In [ ]:
countries['gdp_per_cap'] = countries['gdp_md_est'] / countries['pop_est'] * 100

现在我们可以用这一列对多边形上色:

In [ ]:
ax = countries.plot(figsize=(15, 15), column='gdp_per_cap')
ax.set_axis_off()

In [ ]:
ax = countries.plot(figsize=(15, 15), column='gdp_per_cap', scheme='quantiles', legend=True)
ax.set_axis_off()

#### 合并 dataframes 到一个图表

`.plot` 方法返回一个 matplotlib 轴对象, 可以通过 `ax=` 关键词向图表添加额外的层来复用:

In [ ]:
ax = countries.plot(figsize=(15, 15))
cities.plot(ax=ax, color='red', markersize=10)
ax.set_axis_off()

In [ ]:
ax = countries.plot(edgecolor='k', facecolor='none', figsize=(15, 10))
rivers.plot(ax=ax)
cities.plot(ax=ax, color='C1')
ax.set(xlim=(-20, 60), ylim=(-40, 40))

## 使用 `geoplot`

`geoplot` 包相对于 `.plot()` 方法对 GeoDataFrames 提供一些补充功能:

- 高级绘图 API (具有更多的图表类型)
- 原生投影支持

https://residentmario.github.io/geoplot/index.html

In [ ]:
import geoplot
import geoplot.crs as gcrs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={
    'projection': gcrs.Orthographic(central_latitude=40.7128, central_longitude=-74.0059)
})
geoplot.choropleth(countries, hue='gdp_per_cap', projection=gcrs.Orthographic(), ax=ax,
                   cmap='magma', linewidth=0.5, edgecolor='white', k=None)
ax.set_global()
ax.outline_patch.set_visible(True)
#ax.coastlines()

## 使用 `cartopy`

Cartopy 基于 matplotlib 制图库, `geoplot` 基于它来提供投影.

http://scitools.org.uk/cartopy/docs/latest/index.html

下面的例子来自文档: http://geopandas.readthedocs.io/en/latest/gallery/cartopy_convert.html#sphx-glr-gallery-cartopy-convert-py

In [ ]:
from cartopy import crs as ccrs

In [ ]:
# 定义 CartoPy 坐标参考系统对象.
crs = ccrs.AlbersEqualArea()

# 这个对象可以转换为 `proj4` 字符串/字典来兼容 GeoPandas
crs_proj4 = crs.proj4_init
countries_ae = countries.to_crs(crs_proj4)

# GeoPandas 绘制出来的图表使这样的
countries_ae.plot()

## 线上交互图表

现在有许多面向线上的交互式图表包可以处理地理信息数据，一些包的例子:

- Bokeh: https://bokeh.pydata.org/en/latest/docs/gallery/texas.html
- GeoViews (other interface to Bokeh/matplotlib): http://geo.holoviews.org
- Altair: https://altair-viz.github.io/gallery/choropleth.html
- Plotly: https://plot.ly/python/#maps
- ...

另一个流行的在线地图 javascript 库是 [Leaflet.js](https://leafletjs.com/), 他有 [folium](https://github.com/python-visualization/folium) 和 [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) 包的 python 绑定.

一个 folium 的例子:

In [ ]:
import folium

In [ ]:
m = folium.Map([48.8566, 2.3429], zoom_start=6, tiles="OpenStreetMap")
folium.GeoJson(countries.to_json()).add_to(m)
m

In [ ]:
m = folium.Map([48.8566, 2.3429], zoom_start=6, tiles="OpenStreetMap")
folium.GeoJson(cities.to_json()).add_to(m)
m